In [2]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda:1')

In [3]:
path = '/home/work/nlos/dacon/'
train_y = pd.read_csv(path +"open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (512, 512),interpolation = cv2.INTER_AREA)
    return img

train_png = sorted(glob(path + 'open/train/*.png'))
test_png = sorted(glob(path + 'open/test/*.png'))

train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

np.save(path + 'train_imgs_512', np.array(train_imgs))
np.save(path + 'test_imgs_512', np.array(test_imgs))

In [4]:
train_imgs = np.load(path + 'train_imgs_512.npy')
test_imgs = np.load(path + 'test_imgs_512.npy')

In [5]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('tf_efficientnet_b7', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('tf_efficientnet_b7', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [6]:
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [7]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [8]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [9]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022, shuffle=True)
batch_size = 16
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
    print("----------fold_{} start!----------".format(idx))
    t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
    t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

    # Train
    train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

    # Val
    val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
    val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

    gc.collect()
    torch.cuda.empty_cache()
    best=0

    model = Network().to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()  

    best_f1 = 0
    early_stopping = 0
    for epoch in range(epochs):
        start=time.time()
        train_loss = 0
        train_pred=[]
        train_y=[]
        model.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)


            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()/len(train_loader)
            train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            train_y += y.detach().cpu().numpy().tolist()
        train_f1 = score_function(train_y, train_pred)
        state_dict= model.state_dict()
        model.eval()
        with torch.no_grad():
            val_loss = 0 
            val_pred = []
            val_y = []


        for batch in (val_loader):
            x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
            y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred_val = model(x_val)
            loss_val = criterion(pred_val, y_val)

            val_loss += loss_val.item()/len(val_loader)
            val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
            val_y += y_val.detach().cpu().numpy().tolist()
        val_f1 = score_function(val_y, val_pred)

        
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        
        if val_f1 > best_f1:
            best_epoch = epoch
            best_loss = val_loss
            best_f1 = val_f1
            early_stopping = 0

            torch.save({'epoch':epoch,
                        'state_dict':state_dict,
                        'optimizer': optimizer.state_dict(),
                        'scaler': scaler.state_dict(),
                 }, path +'b7_512_model_{}.pth'.format(idx))
            print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
        else:
            early_stopping += 1
            # Early Stopping
        if early_stopping == 20:
            break

----------fold_0 start!----------


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b7_ra-6c08e654.pth" to /home/work/.cache/torch/hub/checkpoints/tf_efficientnet_b7_ra-6c08e654.pth


epoch : 1/70    time : 250s/17231s
TRAIN loss : 0.94196    f1 : 0.27240
Val loss : 0.47311    f1 : 0.40396
-----------------SAVE:1 epoch----------------
epoch : 2/70    time : 199s/13544s
TRAIN loss : 0.42969    f1 : 0.51438
Val loss : 0.33368    f1 : 0.58627
-----------------SAVE:2 epoch----------------
epoch : 3/70    time : 193s/12935s
TRAIN loss : 0.27892    f1 : 0.64860
Val loss : 0.29562    f1 : 0.67576
-----------------SAVE:3 epoch----------------
epoch : 4/70    time : 197s/13001s
TRAIN loss : 0.20088    f1 : 0.77730
Val loss : 0.34773    f1 : 0.70293
-----------------SAVE:4 epoch----------------
epoch : 5/70    time : 192s/12505s
TRAIN loss : 0.16500    f1 : 0.78881
Val loss : 0.22464    f1 : 0.72374
-----------------SAVE:5 epoch----------------
epoch : 6/70    time : 191s/12226s
TRAIN loss : 0.13344    f1 : 0.85716
Val loss : 0.20481    f1 : 0.73020
-----------------SAVE:6 epoch----------------
epoch : 7/70    time : 187s/11766s
TRAIN loss : 0.09576    f1 : 0.89142
Val loss :

epoch : 37/70    time : 120s/3951s
TRAIN loss : 0.02550    f1 : 0.97920
Val loss : 0.16500    f1 : 0.80182
epoch : 38/70    time : 120s/3837s
TRAIN loss : 0.01845    f1 : 0.97732
Val loss : 0.24546    f1 : 0.79961
epoch : 39/70    time : 110s/3408s
TRAIN loss : 0.10364    f1 : 0.89643
Val loss : 0.24929    f1 : 0.78571
epoch : 40/70    time : 116s/3469s
TRAIN loss : 0.03983    f1 : 0.96313
Val loss : 0.16540    f1 : 0.79856
epoch : 41/70    time : 121s/3511s
TRAIN loss : 0.03034    f1 : 0.97426
Val loss : 0.19747    f1 : 0.80152
epoch : 42/70    time : 130s/3653s
TRAIN loss : 0.04422    f1 : 0.97016
Val loss : 0.18134    f1 : 0.77563
epoch : 43/70    time : 119s/3222s
TRAIN loss : 0.03113    f1 : 0.96361
Val loss : 0.19009    f1 : 0.81404
epoch : 44/70    time : 133s/3445s
TRAIN loss : 0.04356    f1 : 0.95587
Val loss : 0.16235    f1 : 0.83003
epoch : 45/70    time : 127s/3173s
TRAIN loss : 0.03494    f1 : 0.97479
Val loss : 0.25174    f1 : 0.82103
epoch : 46/70    time : 132s/3174s
TR

KeyboardInterrupt: 

In [10]:
pred_ensemble = []
batch_size = 32
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(3):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'b7_512_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [11]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1]) + np.array(pred_ensemble[2]))/3
f_pred = np.array(pred).argmax(1).tolist()

In [12]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [13]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [14]:
submission.to_csv(path + "b7_512_ensemble.csv", index = False)